# Imports

In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sns
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

#from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import OneHotEncoder

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.3.0
Sklearn  0.24.2


# Load Data

In [2]:
df = pd.read_csv('./train.csv', index_col='Id')
df_test = pd.read_csv('./test.csv', index_col='Id')

In [3]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [4]:
df_test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml


# Check Missing

In [29]:
total = df.isnull().sum().sort_values(ascending = False)
total = total[total !=0].dropna() 

percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)
percent = percent[percent !=0].dropna()

merge = pd.concat([total, percent], axis = 1, keys=['Total', 'Percent'])


# for i in merge.iterrows():
#     if i == 0:
#         df.drop(i)

#del_zero_rows(merge['Total'])

pd.set_option('display.max_rows', None)

merge

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
GarageCond,81,0.055479
GarageType,81,0.055479
GarageFinish,81,0.055479


In [6]:
pd.reset_option('^display.', silent=True)

# Feature Extraction

# x & y

## Columns to drop:
- PoolQC
- MiscFeature
- Alley
- Fence
- FireplaceQu

In [7]:
x = df.drop(columns = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'])
y = df['SalePrice']

x_test = df_test.drop(columns = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'])

# Validation Stratergy

In [8]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    #stratify=y,
    random_state=1
)

# Prepro Pipeline

In [9]:
num_cols, cat_cols = [], []

for col in x.columns:
    if x[col].dtype == object:
        cat_cols.append(col)
    else:
        num_cols.append(col)

print('There are', len(cat_cols), 'categorical columns.')
print('There are', len(num_cols), 'numerical columns.')

There are 38 categorical columns.
There are 37 numerical columns.


In [10]:
num_cols.remove('MSSubClass')
num_cols.remove('OverallQual')
num_cols.remove('OverallCond')

In [11]:
cat_cols.append('MSSubClass')
cat_cols.append('OverallQual')
cat_cols.append('OverallCond')

In [12]:
print('There are', len(cat_cols), 'categorical columns.')
print('There are', len(num_cols), 'numerical columns.')

There are 41 categorical columns.
There are 34 numerical columns.


In [13]:
num_preprocessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer()),
    ('normalize', QuantileTransformer())
])

cat_preprocessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer()),
    ('onehot', OneHotEncoder())
])

prepro = compose.ColumnTransformer(transformers=[
    ('nums', num_preprocessing, num_cols),
    ('cat', cat_preprocessing, cat_cols)
])

prepro

ColumnTransformer(transformers=[('nums',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('normalize',
                                                  QuantileTransformer())]),
                                 ['LotFrontage', 'LotArea', 'YearBuilt',
                                  'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
                                  'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
                                  '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
                                  'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
                                  'FullBath', 'HalfBath', 'BedroomAb...
                                  'LandContour', 'Utilities', 'LotConfig',
                                  'LandSlope', 'Neighborhood', 'Condition1',
                                  'Condition2', 'BldgType', 'HouseStyle',
                                  'RoofStyle', 'RoofMatl', 'Exterior1st',
                                  'Exterior2nd', 'MasVnrType', 'ExterQual',
                                  'ExterCond', 'Foundation', 'BsmtQual',
                                  'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Heating', 'HeatingQC',
                                  'CentralAir', 'Electrical', 'KitchenQual', ...])])

In [14]:
x_train_prepro = prepro.fit_transform(x_train)
x_val_prepro = prepro.transform(x_val)

x_test_prepro = prepro.transform(x_test)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'RL'

# Model Pipeline

Regression Problem

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor()
model.fit()

# Cross Validation